In [5]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [6]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory automation' # topic of the search
dataset = '&sp=2655' # access dataset from mergeflow, in this case scientific publications
rows = str(5) # first 5 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Answering questions on the results with OpenAI GPT

In [7]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

       For the problem you found, please only output the following: 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 250-300 words)
        - "User group": Which user group is impacted by this problem the most (please write 20-50 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 80-100 words)
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem:

Title: Advanced Wastewater Reuse

Explanation: In the realm of factory automation, particularly within industries generating high volumes of wastewater, such as coal-fired power plants or chemical manufacturing, there's a growing need for innovative, cost-effective, and environmentally sustainable wastewater treatment and reuse solutions. With technological advancements in reverse osmosis (RO) and mechanical vapor recompression (MVR) highlighting the potential for significant operational cost reductions and improved water resource management, there’s an adjacent possible innovation area in enhancing these technologies for broader applications. The integration of smart automation and real-time monitoring systems into RO-MVR processes could allow for adaptive control strategies, optimizing performance based on fluctuating wastewater characteristics and operational demands. This approach could lead to even greater efficiencies, reduced environmental impact, and compliance with i

Problem:

- "Title": Adaptive Energy Consumption

- "Explanation": In the context of factory automation, a significant emerging challenge is adaptive energy consumption, especially in the wake of integrating renewable energy sources (RES) with industrial processes. The intermittent nature of RES, such as solar and wind power, coupled with the industrial sector's substantial energy demands, creates a complex scenario for energy management. This situation is further complicated by the global push towards reducing carbon footprints and increasing energy efficiency within industries. As factories aim to become more sustainable, the need to adapt their energy consumption to the fluctuating availability of RES becomes paramount. This requires not only smart energy management systems that can dynamically adjust operations based on energy availability but also a shift in how industrial processes are planned and executed. The challenge is to develop systems that can predict energy availability 

In [8]:
# second agent: 

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria.
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)
        
    Be very stringent with your ratings:
        1: Does not meet expectations
        2: Meets expectations adequately
        3: Exceeds expectations significantly
    
    A rating of 3 should be reserved for exceptional performance in the given criterion.

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 50 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


Problem rating:

Problem: Advanced Wastewater Reuse

- Size of user group: 2. The target group consists of industrial facility operators in sectors with significant wastewater output, which is a sizable but not the largest group in factory automation.
- Relevancy of pain point: 3. Highly relevant to the industry, as wastewater management is a critical issue for many factory automation users, particularly in heavy industries.
- How specific is the problem: 3. The problem is specifically tailored to the needs of factory automation in industries with significant wastewater challenges, focusing on innovative and sustainable treatment solutions.

Overall score: 8/9.

Justification: The problem of advanced wastewater reuse is highly relevant and specific to the user group of industrial facility operators, addressing a significant pain point in the sector.

---

Problem rating:

Title: Adaptive Energy Optimization

- Size of user group: 3 (The user group includes a large number of factory man

In [9]:
# third agent to give best problem 

user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
   Please output the problem with the highest overall rating in the {rated_problems_list}, including the following bullet points: 
       - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 100-150 words)
        - "User group": Which user group is impacted by this problem the most (please write 10-30 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 40-50 words)
 
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print(completion.choices[0].message.content)
print("\n---\n")




- **Title**: Skilled Labor Shortage
- **Explanation**: The rapid advancement in factory automation and robotics has led to a critical shortage of skilled labor. This gap is due to the evolving complexity of systems outpacing current educational and vocational training programs. Additionally, manufacturing and industrial sectors struggle to attract younger workers, exacerbating the problem as a significant portion of the skilled workforce nears retirement. This shortage hampers operational efficiency, innovation, and the adoption of cutting-edge technologies, creating significant bottlenecks and limiting competitive edge in a rapidly evolving industry landscape.
- **User group**: Factory Managers and Owners
- **Pain point**: The primary pain point for this user group is the operational challenges brought on by the difficulty in finding and retaining workers with the necessary skills. This leads to increased downtime, reduced efficiency, and higher costs related to training or retraining

In [10]:
problems_and_ratings = [a + b for a, b in zip(problems_list, rated_problems_list)]

def print_menu(options):
    for idx, option in enumerate(options, start=1):
        print(f"{idx}. {option}")

def get_user_choice(num_options):
    while True:
        try:
            choice = int(input("Enter your choice (number): "))
            if 1 <= choice <= num_options:
                return choice
            else:
                print(f"Please enter a number between 1 and {num_options}.")
        except ValueError:
            print("Please enter a valid integer.")

def main():
    options = problems_and_ratings
    print_menu(options)
    choice = get_user_choice(len(options))
    print(f"You chose: {options[choice - 1]}")
    return choice

if __name__ == "__main__":
    chosen_index = main() -1
    chosen_problem = problems_list[chosen_index]
    print("Your choice is:\n" + chosen_problem)

1. Title: Advanced Wastewater Reuse

Explanation: In the realm of factory automation, particularly within industries generating high volumes of wastewater, such as coal-fired power plants or chemical manufacturing, there's a growing need for innovative, cost-effective, and environmentally sustainable wastewater treatment and reuse solutions. With technological advancements in reverse osmosis (RO) and mechanical vapor recompression (MVR) highlighting the potential for significant operational cost reductions and improved water resource management, there’s an adjacent possible innovation area in enhancing these technologies for broader applications. The integration of smart automation and real-time monitoring systems into RO-MVR processes could allow for adaptive control strategies, optimizing performance based on fluctuating wastewater characteristics and operational demands. This approach could lead to even greater efficiencies, reduced environmental impact, and compliance with increasi

Enter your choice (number): 2
You chose: Title: Adaptive Energy Optimization

Explanation: In the rapidly evolving landscape of factory automation, a significant yet under-addressed problem lies in the realm of energy consumption and optimization. As factories grow increasingly automated and reliant on sophisticated machinery and systems, the demand for energy surges, highlighting inefficiencies in current energy management practices. Technological advancements in IoT (Internet of Things) and AI (Artificial Intelligence) have paved the way for smarter, more connected factories. These technologies offer unprecedented data collection and analytics capabilities, enabling real-time monitoring and control of energy usage across various systems. However, the potential for these technologies to dynamically adapt energy consumption in response to fluctuating energy prices, production demands, and renewable energy availability remains largely untapped. This gap represents an adjacent possible, 

# Getting data from Mergeflow's API

In [11]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(20) # first 20 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 19 documents.


In [12]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


HuLoop Automation, a pioneering company in the realm of AI-driven intelligent automation, has successfully completed a significant financial milestone by securing a $5 million seed funding round. This financial injection comes from a consortium of investors led by Moneta Ventures, and includes contributions from several other entities including Rebellion Ventures, Growth Factory, Vibranium VC, Nurture Ventures, Insta VC, among others. The capital raised in this round is earmarked for accelerating HuLoop's mission to transform the automation sector. By leveraging its advanced AI technologies, HuLoop aims to make automation processes not only more efficient and quicker but also more accessible and cost-effective for its expanding client base. This client list encompasses businesses in sectors such as financial services and retail/Consumer Packaged Goods (CPG). The successful closure of this seed funding round marks a pivotal step for HuLoop Automation in its quest to redefine the landsca

Cellares, a company specializing in cell therapy manufacturing based in South San Francisco, California, has successfully raised $255 million in Series C funding. The funding round was led by Koch Disruptive Technologies, with significant contributions from Bristol Myers Squibb, DFJ Growth, Willett Advisors, and other existing investors such as Eclipse, Decheng Capital, and 8VC. This infusion of capital marks a significant milestone for Cellares, which operates as an integrated development and manufacturing organization (IDMO).

The company is at the forefront of advancing cell therapy manufacturing through the development and operation of integrated technologies. Its flagship technology, the Cell Shuttle, offers a comprehensive solution by integrating all necessary technologies for cell therapy manufacturing into a single, flexible platform. This platform is designed to provide end-to-end automation, significantly streamlining the manufacturing process.

Currently, Cellares is operati

Numbers Station, a pioneering firm based in Menlo Park, California, has successfully raised $17.5 million in funding to advance its innovative intelligent data stack automation platform. This financial milestone was achieved through a combination of a $12.5 million Series A funding round led by Madrona, complemented by contributions from Norwest Venture Partners, Factory, and a group of esteemed angel investors including Cloudera Co-founder Jeff Hammerbacher. This Series A funding comes on the heels of a $5 million seed funding round, culminating in a total investment of $17.5 million. In alignment with this investment, Madrona's Managing Director, Tim Porter, will be joining the board of Numbers Station.

Established in 2021 under the leadership of CEO Chris Aberger, Numbers Station has set itself apart by offering an intelligent data stack automation platform. This platform is distinctively built upon proprietary foundation model technology, designed to democratize data automation. I

Worlds Enterprises, Inc., a pioneering company in the development of a 4D infrastructure known as the Industrial Metaverse, has successfully closed a Series A1 funding round, amassing $21.2 million. This round was spearheaded by Moneta Ventures and saw contributions from several existing investors, including Align Capital, Green Park & Golf Ventures, Chevron Technology Ventures, Piva Capital, PerotJain, and Capital Factory. The influx of funds is earmarked for further development of the company’s AI platform, expansion of go-to-market partnerships, and broadening the adoption of its technology among industrial entities.

The Worlds Industrial Metaverse platform is at the forefront of integrating AI-based automation into the foundational operations of large-scale industrial firms. It leverages a live digital twin, encompassing IoT sensors, personnel, and processes, enabling organizations to visualize and revamp their operations in previously unattainable ways. Dave Copps, CEO of Worlds,

CloudNC, a pioneering company in the field of factory automation, has successfully raised $45 million in Series B funding. This financial boost, led by Autodesk and supported by Lockheed Martin, British Patient Capital, and other returning investors, aims to further develop CloudNC's groundbreaking AI and software-as-a-service (SaaS) technology. This technology is set to revolutionize precision manufacturing by making it autonomous, thus enhancing its efficiency, speed, and reliability.

The company's innovative SaaS product promises to simplify the manufacturing process significantly. It enables users, ranging from large corporations to SMEs and individuals, to instruct CNC machines to manufacture precision parts with just a single click. This development is expected to democratize precision manufacturing, making it more accessible and manageable, thereby addressing a crucial $200 billion industry challenge.

Precision manufacturing is an essential process across various industries, i

RIOS Intelligent Machines, Inc., a pioneering company specializing in the development of AI-powered robotic workcells aimed at factory automation, has successfully secured $28 million in Series A funding. This significant financial milestone is a combination of equity funding and debt financing. This recent financial infusion boosts RIOS' total capital to $33 million, factoring in a previous $5 million raised during a venture round in 2020.

The recent Series A funding round was notably oversubscribed, indicating strong investor confidence in RIOS' vision and technology. It was led by Main Sequence, a new investor with a focus on deep technology. Other notable participants in this round included Yamaha Motor Ventures, Orbit Venture Partners, and Hypertherm Ventures, alongside existing investors such as Valley Capital Partners, Morpheus Ventures, and Grit Ventures, all of whom demonstrated their continued support and belief in the company's potential.

This financial achievement undersc

In [13]:
Problem = chosen_problem

print(Problem)

Title: Adaptive Energy Optimization

Explanation: In the rapidly evolving landscape of factory automation, a significant yet under-addressed problem lies in the realm of energy consumption and optimization. As factories grow increasingly automated and reliant on sophisticated machinery and systems, the demand for energy surges, highlighting inefficiencies in current energy management practices. Technological advancements in IoT (Internet of Things) and AI (Artificial Intelligence) have paved the way for smarter, more connected factories. These technologies offer unprecedented data collection and analytics capabilities, enabling real-time monitoring and control of energy usage across various systems. However, the potential for these technologies to dynamically adapt energy consumption in response to fluctuating energy prices, production demands, and renewable energy availability remains largely untapped. This gap represents an adjacent possible, where slight advancements in AI algorithm

# Answering questions on the results with OpenAI GPT

In [14]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the Walt Disney Method: 
        It is a tool for brainstorming ideas. It involves using 3 sequential roles, namely the Dreamer, the Realist, and the Critic.
        The Dreamer: you focus on free association and brainstorming of ideas. Anything goes here. It's your chance to let your imagination run wild! Be enthusiastic and approach the task with questions such as “What would be nice?”, “What would be the optimal situation?” or “What would I wish for?” Assume that everything is possible.
        The Realist: Be realistic and decide which of your ideas are actually possible or practical. What would you need to do in the real world in order to make them happen? Take a pragmatic approach and ask which materials, which knowledge, which techniques there are with which the thoughts of the dreamer can be realised.
        The Critic: Examine proposals and its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. It is about identifying opportunities and risks, pointing out limits and gaps, and expressing constructive and always positive criticism. Try to identify possible sources of error.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
dreamer_solutions = {}
i = 0

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Imagine you are the Dreamer. 
    
    Take a deep breath and work on this problem step by step. 

    
    Please develop 5 really specific ideas/visions on how this problem can be solved. The solutions should consider the following points:
    - You can let your imagination run wild and you do not care about possible limitations. It's more important the idea is new then it can really work.  
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    The solution should contain the following contents:
    1) Your solution in max 3 words (title)
    2) Your solution summarized in 3 sentences.
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) Why the solution is kind of new (not existing on the market)
    5) Please provide some sources of inspiration that you used from the document excerpts.


    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
dreamer_solutions[i] = completion.choices[0].message
i += 1


#Realist prompt (idea: evaluate if it is possible and what would be needed)
realist_solutions = {}

user_prompt1 = f"""

    Imagine you are the Realist. 
    
    Take a deep breath and work on this problem step by step. 

    
    Consider the following solutions: {dreamer_solutions}.
    
    Please give me the 3 clearly speperated pieces of information for each of the solutions (in total 50 words max):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 1 detail": What would you need to do in the real world in order to make the solution happen? Answer which knowledge, which techniques there are with which the solutions can be realised. Explain in 5 sentences. 
    4) "Evaluation criteria 1": Is the solution actually possible? Answer can be 1,2 or 3 (1 = not possible, 2 = maybe possible, 3 = possible) (be the most critical person ever. Only give max. 1 of the 3 solution a 3)

    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
realist_solutions[0] = completion.choices[0].message


#Critics prompt (idea: evaluate if there are a lot potential sources of error)
critics_solutions = {}

user_prompt2 = f"""

    Imagine you are the Critic. 
    
    Consider the following solutions: {realist_solutions}.
    
    Take a deep breath and work on this problem step by step. 
    
    Please give me the following 4 clearly seperated pieces of information for each solution (40 words in total):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 2 details": Identify possible sources of error. Look for risks, pointing out limits and gaps, and expressing constructive criticism. Examine its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. (1 sentence only)
    4) "Evaluation criteria 2": Does it have many potential sources of errors? Answer can be 1,2 or 3 (1 = many sources of error, 2 = some sources of error, 3 = few sources of error) (be the most critical person ever. Only give max. 1 of the 3 solution a 3) (1 sentence)
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
critics_solutions[0] = completion.choices[0].message


#Summary

summary_solutions = {}

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas: {dreamer_solutions}. 
    For each idea you considered how feasible it is {realist_solutions}. 
    Last, you evaluated the amount of potential sources of errors: {critics_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (3 words max)
    “Solved Problem”: 1 or 2 sentence summary of the problem that is being solved 
    “Solution”: Provide a summary of the solution and how it works (in 4-5 sentences). Be dilligent on this task and make sure yo use the length limit. 
    “Impact”: How does the solution solve the specific problem (in 2-3 sentences)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution (in 4-5 sentences)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions[0] = completion.choices[0].message



### 1. Energy Flux Network

**Solution Summary:** 
The Energy Flux Network leverages a decentralized blockchain platform to dynamically manage and optimize energy distribution and consumption across factories. By employing smart contracts, the system automatically adjusts energy usage based on real-time production demands, renewable energy availability, and energy price fluctuations. This innovation not only optimizes energy consumption for cost and efficiency but also promotes the use of renewable energy sources through incentivization.

**Customer Journey:**
Factory managers would participate in the Energy Flux Network by installing IoT devices that monitor energy usage and production metrics in real-time. These devices communicate with the blockchain platform, where AI algorithms analyze the data to optimize energy distribution and consumption across the network. Managers can track energy savings, cost reductions, and carbon footprint reduction through a dashboard, benefiting from a

### 1. Energy Flux Network
- **Title:** Energy Flux Network
- **Summary:** This solution focuses on using a decentralized blockchain platform for dynamic energy management, leveraging smart contracts to adjust energy usage based on factors like renewable energy availability, promising optimized and environmentally friendly consumption.
- **Evaluation criteria 2 details:** The reliance on a cooperative network for energy data sharing may face resistance from factories concerned about data privacy and competitive advantages.
- **Evaluation criteria 2:** 2

### 2. Renewable Energy Microgrids
- **Title:** Renewable Energy Microgrids
- **Summary:** It proposes the creation of autonomous renewable energy networks within factories, utilizing AI to predict and optimize energy flow from sources such as solar and wind, ensuring resilience and sustainability.
- **Evaluation criteria 2 details:** The high initial setup cost and technological complexity could deter small and medium factories from a

In [44]:
# Initial rating of the solutions

initial_rating = {}

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. When evaluating the opportunities, you should take the perspective of a potential client of these solutions, active in the specific industry of {query}.
You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Task: Evaluate the provided solutions based on detailed criteria.

Input Text: {summary_solutions}

Instructions:

Extract Solutions: Begin by identifying all solutions within the input text. For each solution, note the following details:
    - Name
    - Solved Problem
    - Solution Description
    - Impact
    - Technology Used
    - Sources
    
Develop Evaluation Criteria: Create 10 specific criteria to assess the solutions. These criteria should enable a clear comparison between the solutions, focusing on aspects such as innovation, scalability, social impact, and cost-effectiveness.

Rate Each Solution: Apply the criteria to each solution, assigning ratings on a scale from 1 to 3 (1=Lowest, 3=Highest). Be stringent with the ratings:
    1: Does not meet expectations
    2: Meets expectations adequately
    3: Exceeds expectations significantly

A rating of 3 should be reserved for exceptional performance in the given criterion.

Then, also output the total score of a solution, out of the maximum of 30 points.

Repeat for All Solutions: Follow the steps above for each solution presented in the input text.

Objective: This structured analysis will facilitate a comprehensive comparison of the solutions, highlighting their strengths and areas for improvement based on the defined criteria.

Create a table which gives an overview of the solutions and their ratings in the different criteria, giving a number to each solution and only mentionning the number in the horizontal axis of the table. The solution name should not be included in the table.

In your output, please ensure that each solution is separated by "\n---\n".
"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



### Extracted Solutions

1. **Energy Flux Network**
   - **Solved Problem**: Dynamic management and optimization of energy distribution and consumption in factories.
   - **Solution Description**: Utilizes a decentralized blockchain platform with IoT devices for real-time energy monitoring and smart contracts for automatic energy usage adjustments.
   - **Impact**: Reduces operational costs and carbon footprint, aligns operations with renewable energy production.
   - **Technology Used**: Blockchain, smart contracts, IoT devices, AI algorithms.
   - **Sources**: Inspired by HuLoop Automation.

2. **Renewable Energy Microgrids**
   - **Solved Problem**: Inefficiency in energy supply by creating autonomous, renewable energy networks within factories.
   - **Solution Description**: Operates independently or alongside traditional grids, using AI for energy needs prediction and management from renewable sources.
   - **Impact**: Reduces energy costs, increases operational resilience against

In [ ]:
# Pick the best three 

user_prompt5 = f"""
Extract and rank the solutions based on their overall ratings from the provided data: {initial_rating}.

Identify the top three solutions with the highest overall ratings out of 30 and sort them in descending order of their ratings.

For each of these top solutions, provide detailed information based on the content from: {summary_solutions}. Ensure comprehensive coverage of the points listed below:

- "Name": The solution's name, limited to a maximum of 5 words.
- "Solved Problem": A concise three-sentence summary of the problem addressed by the solution (minimum 50 words).
- "Solution": A brief summary of the solution, including its mechanism, in 4 to 5 sentences.
- "Impact": Explanation of how the solution addresses the specific problem, in 2 to 3 sentences.
- "Technology": Detailed description of the technologies used, including their application, aimed at a CTO with an engineering background for implementation purposes, in 4-5 sentences.
- "Sources": Mention any specific inspiration sources from {summary_list}, limited to 1 sentence.

Additionally, highlight the rating criteria where each solution excels.

Please separate each solution's details with "---".

"""

# Call the GPT model to generate the completion
completion = openai.chat.completions.create(
    model="gpt-4-0125-preview",
    messages=[
    {"role": "system", "content": system_prompt_2},
    {"role": "user", "content": user_prompt5}
    ],
    temperature=0.7,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

# Correctly access the generated text from the completion response
generated_text = completion.choices[0].message.content

# Split the generated text into individual solutions based on your specified separator ("---")
solutions_list = generated_text.split("---")

# Initialize an empty dictionary to hold your final solutions
final_solutions = {}

# Populate the dictionary with distinct solutions
for index, solution_text in enumerate(solutions_list, start=1):
    if solution_text.strip():  # Ensure the string is not just whitespace
        final_solutions[index] = solution_text.strip()

# Ensure only the top 3 solutions are kept if more were accidentally generated
final_solutions = dict(list(final_solutions.items())[:3])

# 'final_solutions' now contains up to three distinct items as requested

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


In [17]:
# Integrate new mergeflow data for the solutions 

# Now, based on soultions found, we need more info from Mergeflow


boolean_queries = {}

for index, solution_name in final_solutions.items():
    user_prompt6 = f"""
    
    Create a boolean query based on the Name of the solution from {solution_name}.
    
    Please use the following as an example and follow the format: 
    
    Example: for Factory Blockchain Protocol, this would be:

    "factory blockchain protocol" OR "blockchain for manufacturing" OR "blockchain in production" OR "blockchain in factories" OR "smart factory blockchain" OR "industrial blockchain" OR "supply chain blockchain" OR "blockchain logistics" OR "blockchain traceability"
    

    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
        {"role": "system", "content": system_prompt_2},
        {"role": "user", "content": user_prompt6}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )

    # Extracting the content of the message from the first choice in the choices list
    query_content = completion.choices[0].message.content
    
    # Save the generated boolean query for the current solution
    boolean_queries[index] = query_content

    # Print the generated boolean query for the current solution
    print(query_content)
    print("\n----\n")
    

    

"AI Renewable Sourcing" OR "AI for renewable energy in factories" OR "AI predictive renewable energy sourcing" OR "AI-driven energy optimization" OR "renewable energy forecasting for manufacturing" OR "smart factory renewable energy" OR "AI energy management in production" OR "predictive renewable sourcing in factories" OR "machine learning for renewable energy in industry" OR "AI and IoT for energy optimization in manufacturing"

----

"renewable microgrids" OR "renewable energy networks for factories" OR "autonomous energy grids" OR "factory solar energy solutions" OR "wind energy in manufacturing" OR "AI-driven energy management" OR "sustainable factory energy" OR "energy resilience in production" OR "renewable energy installations for industry" OR "predictive analytics in energy supply"

----

"Energy Flux Network" OR "decentralized energy management" OR "blockchain energy optimization" OR "IoT energy monitoring" OR "smart contracts energy" OR "real-time energy adjustment" OR "rene

In [45]:
dataset = '&sp=3814'  # access dataset from mergeflow - in this case industry news
rows = '3'  # first 3 docs only

# Iterate through each boolean query
for solution, query_text in boolean_queries.items():
    # Correctly encode the query
    encoded_query = query_text.replace(" OR ", '%7C').replace('" ', '%22').replace(' "', '%22').replace(' ', '+').replace('"', '%22')

    # Construct the API request URL
    url = f'https://mergeflow.net/api/v1/leonard/search/getContent?q={encoded_query}{dataset}&rows={rows}'
    mergeflow_url = f'https://mergeflow.net/Profiles/leonard/search?date=default3&q={encoded_query}{dataset}&rows={rows}'

    
    # Define the headers to be sent with the request
    headers = {
        'MergeflowNet-Api-Auth-Key': mergeflow_api_key
    }

    # Send the POST request
    response = requests.post(url, headers=headers)

    mergeflow_docs = []

    # Check the response status code and content
    if response.status_code == 200:
        mergeflow_response_json = response.json()
        
        # Extract and store documents information
        for document in mergeflow_response_json['Documents']:
            current_doc = {
                'title': document['Title'],
                'date': document['Date'],
                'url': document['Url'],
                'content': document['Content']
            }
            
            mergeflow_docs.append(current_doc)
            


    # Process or print your results here
    # For example, print titles of documents for this query
    print(f"Results for {solution}:")
    print("\n-\n") 
    print(f"Mergeflow URL:")
    print(mergeflow_url)
    print("\n-\n") 

    for doc in mergeflow_docs:
        print(doc['title'])
        print(doc['url'])
    print("\n----\n")
    

    

Results for 1:

-

Mergeflow URL:
https://mergeflow.net/Profiles/leonard/search?date=default3&q=%22AI+Renewable+Sourcing%22%7C%22AI+for+renewable+energy+in+factories%22%7C%22AI+predictive+renewable+energy+sourcing%22%7C%22AI-driven+energy+optimization%22%7C%22renewable+energy+forecasting+for+manufacturing%22%7C%22smart+factory+renewable+energy%22%7C%22AI+energy+management+in+production%22%7C%22predictive+renewable+sourcing+in+factories%22%7C%22machine+learning+for+renewable+energy+in+industry%22%7C%22AI+and+IoT+for+energy+optimization+in+manufacturing%22&sp=3814&rows=3

-


----

Results for 2:

-

Mergeflow URL:
https://mergeflow.net/Profiles/leonard/search?date=default3&q=%22renewable+microgrids%22%7C%22renewable+energy+networks+for+factories%22%7C%22autonomous+energy+grids%22%7C%22factory+solar+energy+solutions%22%7C%22wind+energy+in+manufacturing%22%7C%22AI-driven+energy+management%22%7C%22sustainable+factory+energy%22%7C%22energy+resilience+in+production%22%7C%22renewable+energy+i